In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from functools import reduce
import matplotlib as plt

In [2]:
#Best Picture Contenders Source: Indie Wire.com

#cast oscar award predictions (source: Indie Wire.com, assumes all frontrunners win awards)
cast_awards=pd.read_csv('./DataSources/2018CastPredictions.csv')
cast_awards

,Title,Best Director,Best Actor,Best Actress,Best Supporting Actor,Best Supporting Actress
0,Roma,1,0,0,0,0
1,The Favourite,1,0,1,0,1
2,If Beale Street Could Talk,0,0,1,0,1
3,First Man,1,1,0,0,1
4,The Front Runner,0,0,0,0,0
5,A Star Is Born,1,1,1,1,0
6,Black Panther,1,0,0,1,0
7,Green Book,0,0,0,1,0
8,BlacKkKlansman,0,0,0,0,0
9,Can You Ever Forgive Me?,0,0,1,1,0


In [3]:
#OMBD Calls - done on movies list
theMoviesDF_OMBD = pd.read_csv('./DataSources/theContendersDF_OMBD.csv')
theMoviesDF_OMBD

,Actors,Awards,Box Office,Country,Director,Genre,Rated,Title,Writer(s),Year,imdbRating
0,"Yalitza Aparicio, Marina de Tavira, Diego Cort...",NaN,NaN,"USA, Mexico",Alfonso Cuarón,Drama,NaN,Roma,Alfonso Cuarón,2018,7.9
1,"Olivia Colman, Rachel Weisz, Emma Delves, Faye...",NaN,NaN,"Ireland, UK, USA",Yorgos Lanthimos,"Biography, Drama",R,The Favourite,"Deborah Davis, Tony McNamara",2018,8.1
2,"KiKi Layne, Stephan James, Regina King, Colman...",NaN,NaN,USA,Barry Jenkins,"Crime, Drama, Romance",NaN,If Beale Street Could Talk,"Barry Jenkins (written for the screen by), Jam...",2018,6.8
3,"Ryan Gosling, Claire Foy, Jason Clarke, Kyle C...",NaN,NaN,USA,Damien Chazelle,"Biography, Drama, History",NaN,First Man,"Josh Singer (screenplay by), James R. Hansen (...",2018,7.7
4,"Hugh Jackman, Vera Farmiga, J.K. Simmons, Mark...",NaN,NaN,USA,Jason Reitman,"Biography, Drama",NaN,The Front Runner,"Matt Bai, Jay Carson, Jason Reitman, Matt Bai ...",2018,6.6
5,"Lady Gaga, Bradley Cooper, Sam Elliott, Andrew...",1 nomination.,NaN,USA,Bradley Cooper,"Drama, Music, Romance",R,A Star Is Born,"Eric Roth (screenplay by), Bradley Cooper (scr...",2018,8.2
6,"Chadwick Boseman, Michael B. Jordan, Lupita Ny...",14 nominations.,"$501,105,037","USA, South Africa, South Korea, Australia",Ryan Coogler,"Action, Adventure, Sci-Fi",PG-13,Black Panther,"Ryan Coogler, Joe Robert Cole, Stan Lee (based...",2018,7.4
7,"Viggo Mortensen, Linda Cardellini, Mahershala ...",NaN,NaN,USA,Peter Farrelly,"Biography, Comedy, Drama",PG-13,Green Book,"Nick Vallelonga, Brian Hayes Currie, Peter Far...",2018,8.1
8,"Alec Baldwin, John David Washington, Isiah Whi...",NaN,NaN,USA,Spike Lee,"Biography, Comedy, Crime, Drama",R,BlacKkKlansman,"Charlie Wachtel, David Rabinowitz, Kevin Willm...",2018,7.6
9,"Melissa McCarthy, Richard E. Grant, Dolly Well...",NaN,NaN,USA,Marielle Heller,"Biography, Comedy, Crime, Drama",R,Can You Ever Forgive Me?,"Nicole Holofcener (screenplay by), Jeff Whitty...",2018,7.6


In [4]:
#TMDB Calls
theMoviesDF_TMDB=pd.read_csv('./DataSources/theContendersDF_TMDB.csv').rename(columns={"Name":"Title", "Release Date": "Release_Date"})       
theMoviesDF_TMDB

,Title,Revenue,Budget,Runtime,Vote Avg,Vote Count,Popularity,Genre,Release_Date
0,Roma,0,0,134,8.7,17,5.243,Drama,2018-08-25
1,The Favourite,0,0,120,8.0,17,14.494,Drama,2018-11-20
2,If Beale Street Could Talk,0,0,117,0.0,2,10.871,Drama,2018-11-30
3,First Man,94140236,70000000,141,7.2,833,99.314,History,2018-10-11
4,The Front Runner,674563,0,112,5.5,7,4.898,Drama,2018-11-07
5,A Star Is Born,340740907,36000000,135,7.4,1824,101.953,Drama,2018-10-03
6,Black Panther,1346739107,200000000,134,7.3,9125,52.407,Action,2018-02-13
7,Green Book,4409401,0,130,8.2,18,13.276,Drama,2018-11-16
8,BlacKkKlansman,48271960,15000000,135,7.4,1037,54.589,Crime,2018-07-30
9,Can You Ever Forgive Me?,0,0,107,7.2,14,5.828,Drama,2018-10-19


# EXTRACTING INFO FROM API CALLS

In [5]:
#DataFrame by Genre (OMBD)
genreMovies=theMoviesDF_OMBD[['Title', 'Genre']]

for i in range(len(genreMovies['Genre'])):
    x=genreMovies['Genre'][i].split(", ")
    genreMovies['Genre'][i]=x

s=genreMovies.apply(lambda x:pd.Series(x['Genre']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Genre'
BP_genres=genreMovies.drop('Genre', axis=1).join(s)
BP_genres=pd.get_dummies(BP_genres, columns=['Genre'])
BP_genres=BP_genres.groupby('Title').sum()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
#Dataframe by IMBD rating (OMBD)
BP_IMBD=theMoviesDF_OMBD[['Title','imdbRating']]

#Dataframe by Rating (OMBD)
BP_Rating_qual=theMoviesDF_OMBD[['Title','Rated']]
#consolidate certain ratings
BP_Rating_qual['Rated']=BP_Rating_qual['Rated'].replace({'GP':'PG',
                                                        'M/PG': 'PG',
                                                        'TV-MA': 'R',
                                                        'TV-PG': 'PG',
                                                        'PASSED':'APPROVED',
                                                        'NOT RATED': 'UNRATED'})
BP_Rating=pd.get_dummies(BP_Rating_qual, columns=['Rated'])

#Dataframe by Country (OMBD)
BP_Country_qual=theMoviesDF_OMBD[['Title','Country']]
BP_Country=pd.get_dummies(BP_Country_qual, columns=['Country'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


array(['Roma', 'The Favourite', 'If Beale Street Could Talk', 'First Man',
       'The Front Runner', 'A Star Is Born', 'Black Panther',
       'Green Book', 'BlacKkKlansman', 'Can You Ever Forgive Me?'],
      dtype=object)

In [7]:
#Dataframe by Revenue and Budget (TMDB)
BP_Revenue_Budget=theMoviesDF_TMDB[['Title','Revenue', 'Budget']]

#Dataframe by Popularity (TMDB)
BP_Popularity = theMoviesDF_TMDB[['Title','Popularity']]

#Dataframe by Run Time (TMDB)
BP_Runtime = theMoviesDF_TMDB[['Title','Runtime']]

In [8]:
#Dataframe for Release Month (TMDB)
BP_RelMonth_qual = theMoviesDF_TMDB[['Title','Release_Date']]
for i in range(len(BP_RelMonth_qual['Release_Date'])):
    x=BP_RelMonth_qual['Release_Date'][i].split("-")
    BP_RelMonth_qual['Release_Date'][i]=x[1]

BP_RelMonth=pd.get_dummies(BP_RelMonth_qual, columns=['Release_Date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#create a spreadsheet of all info (can have string as values)
dfs=[cast_awards, BP_Country_qual, BP_Popularity, BP_Rating_qual, BP_RelMonth_qual, BP_Revenue_Budget, BP_Runtime, BP_genres]
theMoviesDF_qual = reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title'], how='left'), dfs)
theMoviesDF_qual.fillna(0, inplace=True)
theMoviesDF_qual.to_csv('theMovies_qual_contenders.csv', index=False)
theMoviesDF_qual.shape

(10, 23)

In [10]:
#create a spreadsheet of all info (cannot have string as values)
dfs=[cast_awards, BP_Popularity, BP_Rating, BP_RelMonth, BP_Revenue_Budget, BP_Runtime, BP_genres]
theMoviesDF = reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title'], how='left'), dfs)
theMoviesDF.fillna(0, inplace=True)
theMoviesDF.to_csv('theMovies_contenders.csv', index=False)
theMoviesDF.shape

(10, 27)